<a href="https://colab.research.google.com/github/nenuchepanu001/pysparktutorial/blob/main/BankProspects_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://archive.apache.org/dist/spark/spark-3.1.1/spark-3.1.1-bin-hadoop3.2.tgz
!tar xf spark-3.1.1-bin-hadoop3.2.tgz
!pip install -q findspark
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.1-bin-hadoop3.2"
import findspark
findspark.init()

import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()
spark

In [4]:
spark.createDataFrame([{'Google':"Colab","Spark":"Scala"},{"Google":"DataProc","Spark":"Python"}]).show()

+--------+------+
|  Google| Spark|
+--------+------+
|   Colab| Scala|
|DataProc|Python|
+--------+------+



In [8]:
!wget https://raw.githubusercontent.com/futurexskill/bigdata/master/bank_prospects.csv

--2023-07-01 15:21:50--  https://raw.githubusercontent.com/futurexskill/bigdata/master/bank_prospects.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.110.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 306 [text/plain]
Saving to: ‘bank_prospects.csv’

bank_prospects.csv  100%[===================>]     306  --.-KB/s    in 0s      

2023-07-01 15:21:50 (12.9 MB/s) - ‘bank_prospects.csv’ saved [306/306]



In [10]:
df = spark.read.csv('bank_prospects.csv',header=True)
df.show()

+----+------+------+-------+---------+
| Age|Salary|Gender|Country|Purchased|
+----+------+------+-------+---------+
|  18| 20000|  Male|Germany|        N|
|  19| 22000|Female| France|        N|
|  20| 24000|Female|England|        N|
|  21|  null|  Male|England|        N|
|  22| 50000|  Male| France|        Y|
|  23| 35000|Female|England|        N|
|  24|  null|  Male|Germany|        N|
|  25| 32000|Female| France|        Y|
|null| 35000|  Male|Germany|        N|
|  27| 37000|Female| France|        N|
|  27| 37000|Female|unknown|        N|
+----+------+------+-------+---------+



In [11]:
prospectsDF =  df.filter(df.Country != 'unknown')
prospectsDF.show()

+----+------+------+-------+---------+
| Age|Salary|Gender|Country|Purchased|
+----+------+------+-------+---------+
|  18| 20000|  Male|Germany|        N|
|  19| 22000|Female| France|        N|
|  20| 24000|Female|England|        N|
|  21|  null|  Male|England|        N|
|  22| 50000|  Male| France|        Y|
|  23| 35000|Female|England|        N|
|  24|  null|  Male|Germany|        N|
|  25| 32000|Female| France|        Y|
|null| 35000|  Male|Germany|        N|
|  27| 37000|Female| France|        N|
+----+------+------+-------+---------+



In [12]:
prospectsDF.printSchema()

root
 |-- Age: string (nullable = true)
 |-- Salary: string (nullable = true)
 |-- Gender: string (nullable = true)
 |-- Country: string (nullable = true)
 |-- Purchased: string (nullable = true)



In [21]:
from pyspark.sql.types import *
from pyspark.sql.functions import *

In [16]:
bankProspects = prospectsDF.withColumn('age',prospectsDF.Age.cast('int')).withColumn('salary',prospectsDF.Salary.cast('float'))
bankProspects.printSchema()

root
 |-- age: integer (nullable = true)
 |-- salary: float (nullable = true)
 |-- Gender: string (nullable = true)
 |-- Country: string (nullable = true)
 |-- Purchased: string (nullable = true)



In [17]:
bankProspects.show()

+----+-------+------+-------+---------+
| age| salary|Gender|Country|Purchased|
+----+-------+------+-------+---------+
|  18|20000.0|  Male|Germany|        N|
|  19|22000.0|Female| France|        N|
|  20|24000.0|Female|England|        N|
|  21|   null|  Male|England|        N|
|  22|50000.0|  Male| France|        Y|
|  23|35000.0|Female|England|        N|
|  24|   null|  Male|Germany|        N|
|  25|32000.0|Female| France|        Y|
|null|35000.0|  Male|Germany|        N|
|  27|37000.0|Female| France|        N|
+----+-------+------+-------+---------+



In [26]:
mean_age_val = bankProspects.select(mean(bankProspects.age)).collect()
mean_age_sal = bankProspects.select(mean(bankProspects.salary)).collect()

In [23]:
type(mean_age_val)

list

In [28]:
mean_age = mean_age_val[0][0]
print(mean_age)
mean_sal = mean_age_sal[0][0]
mean_sal

22.11111111111111


31875.0

In [29]:
df1 = bankProspects.na.fill(mean_age,['age'])
df1.show()

+---+-------+------+-------+---------+
|age| salary|Gender|Country|Purchased|
+---+-------+------+-------+---------+
| 18|20000.0|  Male|Germany|        N|
| 19|22000.0|Female| France|        N|
| 20|24000.0|Female|England|        N|
| 21|   null|  Male|England|        N|
| 22|50000.0|  Male| France|        Y|
| 23|35000.0|Female|England|        N|
| 24|   null|  Male|Germany|        N|
| 25|32000.0|Female| France|        Y|
| 22|35000.0|  Male|Germany|        N|
| 27|37000.0|Female| France|        N|
+---+-------+------+-------+---------+



In [30]:
df2 = df1.na.fill(mean_sal,['salary'])
df2.show()

+---+-------+------+-------+---------+
|age| salary|Gender|Country|Purchased|
+---+-------+------+-------+---------+
| 18|20000.0|  Male|Germany|        N|
| 19|22000.0|Female| France|        N|
| 20|24000.0|Female|England|        N|
| 21|31875.0|  Male|England|        N|
| 22|50000.0|  Male| France|        Y|
| 23|35000.0|Female|England|        N|
| 24|31875.0|  Male|Germany|        N|
| 25|32000.0|Female| France|        Y|
| 22|35000.0|  Male|Germany|        N|
| 27|37000.0|Female| France|        N|
+---+-------+------+-------+---------+



In [31]:
df2.write.format('csv').save('bankprospects_transformed')